In [24]:
import json
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [25]:
print("Gpu name", tf.config.experimental.list_physical_devices('GPU'))

Gpu name [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [26]:
with open("intents.json") as f:
    intents_json = json.load(f)

In [27]:
intents_json

{'intents': [{'tag': 'pozdrav',
   'patterns': ['bok', 'hej', 'dobar dan', 'dobro jutro'],
   'responses': ['zdravo', 'dan', 'bok']},
  {'tag': 'odzdrav',
   'patterns': ['doviđenja', 'đenja', 'bok'],
   'responses': ['čujemo se', 'bok', 'vidimo se']}]}

In [28]:
patterns = []
classes = []
responses = []
for intent in intents_json["intents"]:
    for pattern in intent["patterns"]:
        patterns.append(pattern)
        classes.append(intent["tag"])
    responses.append(intent["responses"])

In [29]:
patterns

['bok', 'hej', 'dobar dan', 'dobro jutro', 'doviđenja', 'đenja', 'bok']

In [30]:
classes

['pozdrav', 'pozdrav', 'pozdrav', 'pozdrav', 'odzdrav', 'odzdrav', 'odzdrav']

In [31]:
responses

[['zdravo', 'dan', 'bok'], ['čujemo se', 'bok', 'vidimo se']]

In [9]:
unique_classes = []

for class_ in classes:
    if class_ not in unique_classes:
        unique_classes.append(class_)

In [10]:
unique_classes

['pozdrav', 'odzdrav']

In [11]:
le = LabelEncoder()
labels = le.fit_transform(classes)

In [12]:
labels

array([1, 1, 1, 1, 0, 0, 0])

In [13]:
labels.shape

(7,)

In [14]:
labels = labels.reshape(-1, 1)

In [15]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(patterns)
tokens = tokenizer.texts_to_sequences(patterns)

In [16]:
padded_tokens = pad_sequences(tokens)

In [17]:
padded_tokens

array([[0, 1],
       [0, 2],
       [3, 4],
       [5, 6],
       [0, 7],
       [0, 8],
       [0, 1]], dtype=int32)

In [18]:
padded_tokens.shape

(7, 2)

In [19]:
len(tokenizer.word_index)

8

In [20]:
model = keras.Sequential()
model.add(layers.Embedding(len(tokenizer.word_index) + 1, 64, input_length=2))
model.add(layers.MaxPool1D(2))
model.add(layers.Dense(16, activation="relu"))
model.add(layers.Dense(16, activation="relu"))
model.add(layers.Dense(1, activation="sigmoid"))

2022-02-25 11:17:40.396003: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-25 11:17:40.397004: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-25 11:17:40.397307: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-25 11:17:40.397545: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 2, 64)             576       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1, 64)            0         
 )                                                               
                                                                 
 dense (Dense)               (None, 1, 16)             1040      
                                                                 
 dense_1 (Dense)             (None, 1, 16)             272       
                                                                 
 dense_2 (Dense)             (None, 1, 1)              17        
                                                                 
Total params: 1,905
Trainable params: 1,905
Non-trainable params: 0
______________________________________________________

In [22]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [33]:
model.fit(padded_tokens, labels, epochs=20, validation_split=0.2)

Epoch 1/20
1/1 [==============================] - 0s 165ms/step - loss: 0.6519 - accuracy: 1.0000 - val_loss: 0.6837 - val_accuracy: 0.5000
Epoch 2/20
1/1 [==============================] - 0s 22ms/step - loss: 0.6483 - accuracy: 1.0000 - val_loss: 0.6851 - val_accuracy: 0.5000
Epoch 3/20
1/1 [==============================] - 0s 20ms/step - loss: 0.6444 - accuracy: 1.0000 - val_loss: 0.6873 - val_accuracy: 0.5000
Epoch 4/20
1/1 [==============================] - 0s 22ms/step - loss: 0.6403 - accuracy: 1.0000 - val_loss: 0.6900 - val_accuracy: 0.5000
Epoch 5/20
1/1 [==============================] - 0s 24ms/step - loss: 0.6360 - accuracy: 1.0000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 6/20
1/1 [==============================] - 0s 23ms/step - loss: 0.6315 - accuracy: 1.0000 - val_loss: 0.6966 - val_accuracy: 0.5000
Epoch 7/20
1/1 [==============================] - 0s 23ms/step - loss: 0.6270 - accuracy: 1.0000 - val_loss: 0.7003 - val_accuracy: 0.5000
Epoch 8/20
1/1 [==========